# Client


Проверять работу клиентской части будем с помощью того же датасета, но с помощью данных для валидации.

В дальнейшем можно будет передавать произвольные данные на сервер с целью первичной сортировки вопросов.

## 1. Установка необходимых библиотек

Можно скачать датасет напрямую с кагла, но для этого необходимо получить Kaggle API Token, загрузить файл с токеном на Google Colab и запустить код ниже:

In [ ]:
# !pip install kaggle

In [ ]:
# !mkdir ~/.kaggle
# !cp ./kaggle.json ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download imoore/60k-stack-overflow-questions-with-quality-rate

In [ ]:
# !unzip *.zip

Иначе можно скачать отсюда:

https://drive.google.com/file/d/1dSJuZ_WqNUyXywLvAx-5wic0IxsiSrc7/view?usp=sharing

И загрузить на Google Colab

In [ ]:
import pandas as pd
import numpy as np
import requests
import urllib.request
import json

## 2. Проверим работу обращения к серверу

В функции ниже необходимо изменить url сервера при работе через Googe Colab.

In [ ]:
# функция формирует запрос, направляет на сервер и получает ответ в виде предсказания
def send_json(x):
    title, body, tags = x
    #print(description, company_profile, benefits)
    body = {
        'Title': title,
        'Body': body,
        'Tags': tags
        }
    myurl = 'http://83e9-35-236-240-147.ngrok-free.app/' + '/predict' # для работы в google colab. URL сервера каждый раз генерируется новый, его необходимо менять.
    # myurl = 'http://127.0.0.1:5000/' + '/predict' # при запуске на локальной машине
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['Predictions']

In [ ]:
# Пример данных с одним запросом
data = (
    "How to get all the child records from ",
    "I am having 4 different tables like",
    "<oop>"
)

In [ ]:
response = send_json(data)
print('предсказание', response)

предсказание LQ_EDIT


## 3. Сделаем обработку массовых запросов:

Загрузим данные valid.csv

In [ ]:
df_valid = pd.read_csv("./valid.csv")
# df_valid['Y'] = df_valid['Y'].map({'LQ_CLOSE':0, 'LQ_EDIT': 1, 'HQ':2}) # если категории переведены в численный вид
df_valid.head(5)

,Id,Title,Body,Tags,CreationDate,Y
0,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
1,34554721,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,<php><mysql><sql><codeigniter><mysqli>,2016-01-01 08:43:50,LQ_EDIT
2,34555135,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,<python><pandas>,2016-01-01 09:55:22,HQ
3,34555448,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",<sql-server><c#-4.0>,2016-01-01 10:43:45,LQ_EDIT
4,34555752,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,<php>,2016-01-01 11:34:09,LQ_EDIT


In [ ]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            15000 non-null  int64 
 1   Title         15000 non-null  object
 2   Body          15000 non-null  object
 3   Tags          15000 non-null  object
 4   CreationDate  15000 non-null  object
 5   Y             15000 non-null  object
dtypes: int64(1), object(5)
memory usage: 703.2+ KB


In [ ]:
X_val = df_valid[['Title', 'Body', 'Tags']]
y_val = df_valid['Y']

In [ ]:
len(y_val)

15000

## 4. Передаем одиночный запрос из наших данных и посмотрим на ответ

In [ ]:
response = send_json(X_val.iloc[0,:])

In [ ]:
response

'LQ_EDIT'

## 5. Сделаем N запросов и оценим время

In [ ]:
N = 100

In [ ]:
%%time
predictions = X_val[['Title', 'Body', 'Tags']].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 375 ms, sys: 58 ms, total: 433 ms
Wall time: 10.2 s


In [ ]:
predictions.values[:5]

array(['LQ_EDIT', 'LQ_EDIT', 'HQ', 'LQ_EDIT', 'LQ_EDIT'], dtype=object)

## 6. Посчитаем метрику

In [ ]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
      if actual[i] == predicted[i]:
        correct += 1
    return (correct / float(len(actual))) * 100.0

In [ ]:
test_accuracy_N = accuracy_metric(np.array(y_val)[:N], predictions)
test_accuracy_N

82.0

При очень массовой загрузке (в файле 15000 строк) следующая строка иногда выдает ошибку, т.к. сервер не успевает обрабатывать запросы. Возможно стоит добавить задержку.

In [ ]:
%%time
predictions = X_val.apply(lambda row: send_json(row), axis=1)

JSONDecodeError: ignored

In [ ]:
test_accuracy = accuracy_metric(np.array(y_val), predictions)
test_accuracy

## 7. Вывод:

Сервер работает корректно, обрабатывает данные и выдает предсказания по классификации вопросов с точностью предсказаний ~80%.